In [ ]:
from math import factorial, exp
import matplotlib.pyplot as plt
import numpy as np
import csv

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def erlangB(n, A0):
    denom = 0
    for i in range(n+1):
        denom += (A0**i)/(factorial(i))
    E1 = ((A0**n)/factorial(n))/denom
    return E1

# def erlangC(n, A0):
#     denom = ((A0**n)/factorial(n))*(n/(n-A0))
#     for i in range(n):
#         denom += (A0**i)/factorial(i)
#     return ((A0**n)/factorial(n))*(n/(n-A0))/denom

def erlangC(n, A0):
    denomSum = 0
    for i in range(n):
        denomSum += (A0**i)/factorial(i)
    denom = A0**n + factorial(n)*(1-A0/n)*denomSum
    return (A0**n)/denom

def meanCallDelay(n, A0, meanCallDuration, probDelay):
    # print(probDelay)
    return probDelay*meanCallDuration/(n-A0)

def meanQueuedDelay(n, A0, meanCallDuration):
    return meanCallDuration/(n-A0)

def erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=0):
    probDelay = erlangC(n,A0)
    return probDelay*exp(-(n-A0)*allowedDelay/meanCallDuration)


In [ ]:
def trafficSimulation(numChannels, numCalls, meanCallDuration, requeue=False, maxDelay=0, holdingDistrib='exp', repetitions=1):

    callFailRates = np.zeros(repetitions)
    meanCallDelay = np.zeros(repetitions)

    for i in range(repetitions):

        if holdingDistrib == 'exp':
            callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)

        elif holdingDistrib == 'gamma':
            # callDurations = np.random.gamma(shape=1, scale=meanCallDuration, size=numCalls)
            callDurations = np.random.gamma(shape=1.2073, scale=0.03205, size=numCalls)  # shape from paper, scale from paper: 0.03205 = muCHT*(muPaper/bPaper) = muCHT * (29.6121/35.875)

        callStarts = np.random.uniform(size=numCalls)
        # callStarts = np.random.poisson(lam=1/(meanCallDuration*3600), size=numCalls)
        # plt.hist(callStarts, 50)
        # plt.show()
        callStarts.sort()

        # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
        calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

        channelFreeTimes = np.zeros(numChannels)
        time = 0
        callsFailed = 0
        delay = 0
        for call in calls:
            time = call[0]
            channelFound = False

            for j in range(numChannels):
                if not(channelFound):
                    if channelFreeTimes[j] <= time:
                        channelFreeTimes[j] = call[1]
                        channelFound = True

            if channelFound == False:
                if requeue == False:
                    callsFailed += 1

                else:
                    nextCallFinishTime = channelFreeTimes.min()
                    delay += nextCallFinishTime-call[0]
                    if delay>maxDelay:
                        callsFailed += 1
                        indices = np.where(channelFreeTimes == nextCallFinishTime)
                        index = indices[0][0]
                        channelFreeTimes[index] = call[1]

        callFailRates[i] = callsFailed/numCalls
        meanCallDelay[i] = delay/numCalls

    if requeue == False:
        return callFailRates.mean(), np.std(callFailRates, ddof=1)
    else:
        return callFailRates.mean(), np.std(callFailRates, ddof=1), meanCallDelay.mean(), np.std(meanCallDelay, ddof=1)

In [ ]:
meanCallDuration = 2.33/60# 2.33 mins in hrs
n = 57

# minNumCalls = 1
# maxNumCalls = 2000
# numTests = 21
# simulationCallAmounts = np.concatenate((np.linspace(minNumCalls, maxNumCalls, numTests), np.linspace(2500, 6000, 8))).astype(np.int)
simulationCallAmounts = np.linspace(0, 3000, 16).astype(np.int)
# simulationCallAmounts = np.linspace(0, 6000, 31).astype(np.int)
# simulationCallAmounts = np.linspace(1000, 1000, 1).astype(np.int)

repetitionsPerSimulation = 10
print(simulationCallAmounts)

In [ ]:
# CALLS BLOCKED - NO REQUEUING

erlangBs = np.zeros(simulationCallAmounts.shape)
GOS_exp = np.zeros(simulationCallAmounts.shape)
stdDevs_exp = np.zeros(simulationCallAmounts.shape)
GOS_gamma = np.zeros(simulationCallAmounts.shape)
stdDevs_gamma = np.zeros(simulationCallAmounts.shape)

print("# Calls:\t\tErlang B:\t\tSim GOS (exp):\t\tstdDev (exp):\t\tSim GOS (gamma)\t\tstdDev (gamma):")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = offeredTraffic(numCalls, meanCallDuration)
    erlangBs[i] = erlangB(n, A0)*100

    meanBlockingRate_exp, stdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    GOS_exp[i] = meanBlockingRate_exp*100
    stdDevs_exp[i] = stdDev_exp*100

#     meanBlockingRate_gamma, stdDev_gamma = trafficSimulation(n, numCalls, meanCallDuration, holdingDistrib='gamma', repetitions=repetitionsPerSimulation)
#     GOS_gamma[i] = meanBlockingRate_gamma*100
#     stdDevs_gamma[i] = stdDev_gamma*100

    print("{}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls,erlangBs[i], GOS_exp[i], stdDevs_exp[i], GOS_gamma[i], stdDevs_gamma[i]))
trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
plt.figure(figsize=(15,8))
plt.plot(trafficRange, erlangBs, label='Erlang B', color="black")
plt.errorbar(trafficRange, GOS_exp, yerr=stdDevs_exp, label="Monte Carlo (Exponential)", color="red")
plt.fill_between(trafficRange, GOS_exp+stdDevs_exp, GOS_exp-stdDevs_exp, interpolate=True, color="red", alpha=0.15)

# plt.errorbar(trafficRange, GOS_gamma, yerr=stdDevs_gamma, label="Gamma", color="green")
# plt.fill_between(trafficRange, GOS_gamma+stdDevs_gamma, GOS_gamma-stdDevs_gamma, interpolate=True, color="green", alpha=0.2)

plt.xlabel('Offered traffic (Erlangs)')
plt.ylabel('Grade Of Service (%)')
plt.title("GOS vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()


In [ ]:
# WITH REQUEUING

erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays_calc = np.zeros(simulationCallAmounts.shape)

GOS_exp = np.zeros(simulationCallAmounts.shape)
stdDevs_exp = np.zeros(simulationCallAmounts.shape)
meanCallDelays_exp = np.zeros(simulationCallAmounts.shape)
callDelayStdDevs_exp = np.zeros(simulationCallAmounts.shape)

# GOS_gamma = np.zeros(simulationCallAmounts.shape)
# stdDevs_gamma = np.zeros(simulationCallAmounts.shape)
# meanCallDelays_gamma = np.zeros(simulationCallAmounts.shape)
# callDelayStdDevs_gamma = np.zeros(simulationCallAmounts.shape)

allowedDelay=0#20/3600 # in hours

print("# Calls:\t\tErlang C:\t\tSim GOS (exp):\t\tSim GOS stdDev\t\tCalc. delay/call\tSim Mean delay/call\tdelay/call stdDev")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = offeredTraffic(numCalls, meanCallDuration)
    
    erlangCs[i] = erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=allowedDelay)*100
    meanCallDelays_calc[i] = meanCallDelay(n, A0, meanCallDuration, np.clip(erlangCs/100,0,1)[i])

    meanDelayRate_exp, stdDev_exp,  meanCallDelay_exp, callDelayStdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    GOS_exp[i] = meanDelayRate_exp*100
    stdDevs_exp[i] = stdDev_exp*100
    meanCallDelays_exp[i] = meanCallDelay_exp
    callDelayStdDevs_exp[i] = callDelayStdDev_exp

    # meanDelayRate_gamma, stdDev_gamma,  meanCallDelay_gamma, callDelayStdDev_gamma = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, holdingDistrib='gamma', repetitions=repetitionsPerSimulation)
    # GOS_gamma[i] = meanDelayRate_gamma*100
    # stdDevs_gamma[i] = stdDev_gamma*100
    # meanCallDelays_gamma[i] = meanCallDelay_gamma
    # callDelayStdDevs_gamma[i] = callDelayStdDev_gamma

    print("{}\t\t\t{:.3f}\t\t\t{:.3f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, erlangCs[i], GOS_exp[i], stdDevs_exp[i], meanCallDelays_calc[i], meanCallDelays_exp[i], callDelayStdDevs_exp[i]))

trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
plt.figure(figsize=(15,8))

plt.plot(trafficRange, np.clip(erlangCs,0,100), label='Erlang C', color="orange")
plt.plot(trafficRange, erlangBs, label='Erlang B', color="black")

plt.errorbar(trafficRange, GOS_exp, yerr=stdDevs_exp, label="Monte Carlo with requeue (Exponential)", color="red")
plt.fill_between(trafficRange, GOS_exp+stdDevs_exp, GOS_exp-stdDevs_exp, interpolate=True, color="red", alpha=0.15)

# plt.errorbar(trafficRange, GOS_gamma, yerr=stdDevs_gamma, label="Monte Carlo with requeue (Gamma)", color="green")
# plt.fill_between(trafficRange, GOS_gamma+stdDevs_gamma, GOS_gamma-stdDevs_gamma, interpolate=True, color="green", alpha=0.15)
# plt.plot(trafficRange, GOS_simulated, label='Exponential')

plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('Grade Of Service (% probability of a call being delayed more than {} seconds)'.format(maxDelay*3600))
plt.title("GOS vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()


In [ ]:
simulationCallAmounts = np.linspace(800, 1425, 26).astype(np.int)
repetitionsPerSimulation = 50
print(simulationCallAmounts)

erlangCs = np.zeros(simulationCallAmounts.shape)
meanCallDelays_calc = np.zeros(simulationCallAmounts.shape)

meanCallDelays_exp = np.zeros(simulationCallAmounts.shape)
callDelayStdDevs_exp = np.zeros(simulationCallAmounts.shape)

allowedDelay=0#20/3600 # in hours

print("# Calls\t\tTraffic:\tCalc. delay/call\tSim Mean delay/call\tdelay/call stdDev")
for i, numCalls in enumerate(simulationCallAmounts):

    A0 = offeredTraffic(numCalls, meanCallDuration)
    
    erlangCs[i] = erlangC_allowedDelayGOS(n, A0, meanCallDuration, allowedDelay=allowedDelay)
    
    meanCallDelays_calc[i] = meanCallDelay(n, A0, meanCallDuration, erlangCs[i])*3600

    _, _,  meanCallDelay_exp, callDelayStdDev_exp = trafficSimulation(n, numCalls, meanCallDuration, requeue=True, holdingDistrib='exp', repetitions=repetitionsPerSimulation)
    meanCallDelays_exp[i] = meanCallDelay_exp*3600
    callDelayStdDevs_exp[i] = callDelayStdDev_exp*3600
    print("{}\t\t{:.3f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}".format(numCalls, A0, meanCallDelays_calc[i], meanCallDelays_exp[i], callDelayStdDevs_exp[i]))

plt.figure(figsize=(12,6))
trafficRange = offeredTraffic(simulationCallAmounts,meanCallDuration)
# plt.plot(trafficRange, np.clip(meanCallDelays_calc,0,100), label='Erlang C Calculated Delay', color="orange")
plt.errorbar(trafficRange, meanCallDelays_exp, yerr=callDelayStdDevs_exp, label="Monte Carlo with requeue (Exponential)", color="red")
plt.fill_between(trafficRange, meanCallDelays_exp+callDelayStdDevs_exp, meanCallDelays_exp-callDelayStdDevs_exp, interpolate=True, color="red", alpha=0.15)
plt.ylim([0,None])
plt.xlabel('Offered Traffic (Erlangs)')
plt.ylabel('Mean Delay/Call (seconds)')
plt.title("Mean Delay per Call vs Offered Traffic ({} channels)".format(n))
plt.legend()
plt.show()

In [ ]:
with open('block.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["# Calls", "Erlang B:", "Sim GOS (exponential):", "Sim std dev (exponential):", "Sim GOS (gamma)", "Sim std dev (gamma)", "Repetitions per simulation = "+str(repetitionsPerSimulation)])
    for i, numCalls in enumerate(simulationCallAmounts):
        writer.writerow([numCalls, erlangBs[i], GOS_exp[i], stdDevs_exp[i], GOS_gamma[i], stdDevs_gamma[i]])